# Arabic Spoken Digit Recognition

## Load the Datasets

In [19]:
import numpy as np


def read_data(f:str, n_speakers:int, num_times:int=10) -> list[np.ndarray]:
    """
    Read data from the dataset.
    Data is formatted in blocks. Each block has 13 rows and ~30 columns.
    Blocks are separated by an empty row.
    Each column represents one of the 13 cepstral coefficients.
    Each block is a numpy array of shape (13, ~30).

    Parameters
    ----------
    f : str
        Path to the dataset.

    n_speakers : int
        Number of speakers whose voice is recorded in the dataset.

    num_times : int
        Number of times each speaker speaks a given number in the dataset.

    Returns
    -------
    data : list
        List of numpy arrays, where each element is a numpy array.
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] # 10 digits
        |\
        | \
        [0, 1, 3, 4, 5, ..., 660]  # 660 = n_speakers * num_times
        |\
        | \
        np.array([[0, 1, 2, 3, 4, 5, ..., 13],
                  [0, 1, 2, 3, 4, 5, ..., 13],
                  ...
                  [0, 1, 2, 3, 4, 5, ..., 13]])  # 13 = number of cepstral coefficients, as many rows as there are frames (~30)
    """

    data = []  # Initialize an empty list to store the data
    current_digit_data = []  # Initialize a list to store data for the current digit
    current_block = []  # Initialize a list to store data for the current block

    with open(f, 'r') as file:
        for line in file:
            line = line.strip()
            if line:  # Non-empty line
                values = [float(value) for value in line.split()]  # Parse values from the line
                current_block.append(np.array(values).reshape(-1, 13))
            else:  # Empty line (block separator)
                to_add = np.array(current_block).reshape(-1, 13)
                current_digit_data.append(to_add)
                current_block = []  # Reset the current block
                if len(current_digit_data) == n_speakers * num_times:
                    data.append(current_digit_data)
                    current_digit_data = []  # Reset the current_digit_data, move on to the next digit

        # Add last digit's data
        # Edge case: last digit's data is not added in the loop above
        current_digit_data.append(np.array(current_block).reshape(-1, 13))
        data.append(current_digit_data)

    return data

In [20]:
TEST = read_data("data/Test_Arabic_Digit.txt", n_speakers=22)
print(f'Number of digitis in the test dataset: {len(TEST)}')
print(f'Total number of blocks in the test dataset: {sum([len(digit) for digit in TEST])}')
print(f'Number of blocks for each digit: {[len(digit) for digit in TEST]}')
print(f'Sample block shapes for each digit: {[digit[0].shape for digit in TEST]}')

Number of digitis in the test dataset: 10
Total number of blocks in the test dataset: 2200
Number of blocks for each digit: [220, 220, 220, 220, 220, 220, 220, 220, 220, 220]
Sample block shapes for each digit: [(28, 13), (31, 13), (36, 13), (41, 13), (37, 13), (34, 13), (50, 13), (39, 13), (46, 13), (36, 13)]


In [21]:
TRAIN = read_data("data/Train_Arabic_Digit.txt", n_speakers=66)
print(f'Number of digitis in the train dataset: {len(TRAIN)}')
print(f'Total number of blocks in the train dataset: {sum([len(digit) for digit in TRAIN])}')
print(f'Number of blocks for each digit: {[len(digit) for digit in TRAIN]}')
print(f'Sample block shapes for each digit: {[digit[0].shape for digit in TRAIN]}')

Number of digitis in the train dataset: 10
Total number of blocks in the train dataset: 6600
Number of blocks for each digit: [660, 660, 660, 660, 660, 660, 660, 660, 660, 660]
Sample block shapes for each digit: [(38, 13), (32, 13), (35, 13), (38, 13), (31, 13), (29, 13), (42, 13), (31, 13), (44, 13), (34, 13)]


In [22]:
# Supplementary information about how many frames are male for each digit
# This is the sum of the lengths for the first half of the utterances for each digit

def male_frames(data):
    male_frames = []
    for digit in data:
        ld = len(digit)
        half_ld = ld // 2
        s = sum([len(digit[i]) for i in range(half_ld)])
        male_frames.append(s)
    return male_frames

train_male_frames = male_frames(TRAIN)
test_male_frames = male_frames(TEST)

print('Train male frames')
print(train_male_frames)

print('Test male frames')
print(test_male_frames)

Train male frames
[11588, 10803, 12987, 14020, 13018, 10403, 14014, 12253, 15546, 11431]
Test male frames
[3817, 3964, 4786, 4794, 4647, 3508, 5073, 4405, 5736, 3837]


In [23]:
# We need to gather the points from all digits, not just the first one
# Make a list of all the digits -- i.e. append all utterances within each digit
# Each element in the list X is of the form:

# [-----------------13-----------------]
# |
# |
# | n_speakers * ~30 * 10
# |
# |
# |

import pandas as pd


def concatenate(x: list[np.ndarray]) -> list[np.ndarray]:
    '''
    Concatenate all utterances for each digit.
    '''
    to_append = []
    for digit in x:
        digit_utterances = []
        for utterance in digit:
            if utterance.shape[1] == 13:
                digit_utterances.append(utterance)
        to_append.append(digit_utterances)

    return [np.concatenate(to_append_digit, axis=0) for to_append_digit in to_append]

# Train data is all the utterances for each digit concatenated
X_TRAIN = concatenate(TRAIN)

X_TEST = concatenate(TEST)

# Display the shapes of the data as a dataframe
df = pd.DataFrame({'Train': [x.shape for x in X_TRAIN]})
df.style.set_caption("Shape of the datasets for each digit")


,Train
0,"(23344, 13)"
1,"(22652, 13)"
2,"(27938, 13)"
3,"(29406, 13)"
4,"(27555, 13)"
5,"(21631, 13)"
6,"(28991, 13)"
7,"(24924, 13)"
8,"(32860, 13)"
9,"(23955, 13)"


## Create the Model

In [24]:
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KernelDensity
from tqdm import tqdm


class DigitClassifier:
    '''
    Class for a digit classifier using Gaussian Mixture Models and Kernel Density Estimation based on the log-likelihoods.

    Parameters
    ----------
    num_clusters : int
        Number of clusters for each digit.

    bandwidth : float
        Bandwidth for the kernel density estimation.

    cepstral_mask : list
        List of booleans to mask the cepstral coefficients. Defaults to [True] * 13.

    covariance_type : str
        Covariance type for the Gaussian Mixture Model. Defaults to 'full'.
        Other options are 'spherical', 'diag', and 'tied'.
    '''

    def __init__(self, num_clusters: int = 8, bandwidth: float = 0.5, cepstral_mask = [True] * 13, covariance_type: str = 'full'):
        self.num_clusters = num_clusters
        self.covariance_type = covariance_type
        self.GMMs = []
        self.weights = []
        self.cepstral_mask = cepstral_mask
        self.kde = KernelDensity(bandwidth=bandwidth)

    def model_info(self):
        '''
        Print the model information.

        Returns
        -------
        info : str
        '''
        return f'GMM with {self.num_clusters} clusters, cepstral mask {self.cepstral_mask}, and covariance type {self.covariance_type}'
    
    def fit(self, train_data: list[np.ndarray]):
        '''
        Fit the model to the training data.

        Parameters
        ----------
        train_data : list
            List of numpy arrays, where each element is the concatenation of all utterances for a given digit.
        '''

        self.gmms = []

        # Mask the cepstral coefficients by copying and slicing the data
        processed_train_data = []
        for digit in train_data:
            processed_train_data.append(digit[:, self.cepstral_mask])

        # for i in tqdm(range(10), leave=False):
        for i in range(10):
            x = processed_train_data[i]
            gmm = GaussianMixture(n_components=self.num_clusters, random_state=42, covariance_type=self.covariance_type).fit(x)
            self.GMMs.append(gmm)
            self.weights = gmm.weights_

    def predictions(self, x: list[np.ndarray]) -> list[float]:
        '''
        Predict the digit for each utterance in x.

        Parameters
        ----------
        x : list
            List of numpy arrays, where each element is a numpy array of shape (13, ~30).

        Returns
        -------
        predicted_means : list
            List of predicted means for each utterance in x.
        '''

        predicted_means = []

        for gmm in self.GMMs:
            # Compute the log-likelihood of each point in x_digit
            log_likelihood = gmm.score_samples(x)

            # Estimate the PDF of the likelihoods using KernelDensity
            self.kde.fit(log_likelihood.reshape(-1, 1))

            # Print the mean values
            predicted_mean = np.mean(log_likelihood)
            predicted_means.append(predicted_mean)

        return predicted_means
    
    def predict(self, x: list[np.ndarray]) -> int:
        '''
        Predict the digit for the utterance x.

        Parameters
        ----------
        x : list
            List of numpy arrays, where each element is a numpy array of shape (13, ~30).

        Returns
        -------
        predicted_digit : int
            Predicted digit.
        '''
        # If the mask hasn't been applied, apply it
        if x.shape[1]  != sum([1 for i in self.cepstral_mask if i]):
            x = x[:, self.cepstral_mask]
            
        predicted_means = self.predictions(x)
        return np.argmax(predicted_means)
    
    def test_accuracy(self, test_data: list[np.ndarray], verbose: bool = False) -> float:
        '''
        Compute the accuracy of the model on the test data.

        Parameters
        ----------
        test_data : list
            List of numpy arrays, where each element is a numpy array of shape (13, ~30).

        verbose : bool
            Whether to print the accuracy for each digit.

        Returns
        -------
        accuracies : tuple
            Tuple of (overall_accuracy, digit_accuracies).
        '''

        # Again mask the cepstral coefficients by copying and slicing the data
        processed_test_data = []
        for digit in test_data:
            digit_utterances = []
            for utterance in digit:
                # Apply the mask
                digit_utterances.append(utterance[:, self.cepstral_mask])
            processed_test_data.append(digit_utterances)

        total_tested = 0
        correct = 0

        digit_accuracies = []

        # Loop through all different digits
        for digit_num, digit in enumerate(processed_test_data):
            digit_utterances_correct = 0
            total_digit_utterances = len(digit)

            # Loop through all utterances for the current digit
            for utterance in digit:
                predicted_digit = self.predict(utterance)
                if predicted_digit == digit_num:
                    digit_utterances_correct += 1

            if verbose:
                print(f'Accuracy for digit {digit_num}: {digit_utterances_correct / total_digit_utterances}')

            digit_accuracies.append(digit_utterances_correct / total_digit_utterances)

            total_tested += total_digit_utterances
            correct += digit_utterances_correct

        if verbose:
            print(f'Overall accuracy: {correct / total_tested}')

        return correct / total_tested
        

In [25]:
class GenderDigitClassifier(DigitClassifier):

    def __init__(self, num_clusters: int = 8, bandwidth: float = 0.5, cepstral_mask = [True] * 13, covariance_type: str = 'full'):
        super().__init__(num_clusters, bandwidth, cepstral_mask, covariance_type)

    def fit(self, train_data: list[np.ndarray]):
        '''
        Fit the model to the training data.
        Assumes that train_data is actually the train data, as it uses the sizes from before to understand male/female distribution. 

        Parameters
        ----------
        train_data : list
            List of numpy arrays, where each element is the concatenation of all utterances for a given digit.
        '''

        self.gmms = []

        # Mask the cepstral coefficients by copying and slicing the data
        # Also split for gender
        # That is, processed_train_data is of the form [0M, 0F, 1M, 1F, ..., 9M, 9F]
        processed_train_data = []
        for i, digit in enumerate(train_data):
            sliced = digit[:, self.cepstral_mask]

            # Now split by train_male_frames, as in the first train_male_frames rows will be added independently
            processed_train_data.append(sliced[:train_male_frames[i], :])
            processed_train_data.append(sliced[train_male_frames[i]:, :])

        # for i in tqdm(range(20), leave=False):
        for i in range(20):
            x = processed_train_data[i]
            gmm = GaussianMixture(n_components=self.num_clusters, random_state=42, covariance_type=self.covariance_type).fit(x)
            self.GMMs.append(gmm)
            self.weights = gmm.weights_
    
    def predict(self, x: list[np.ndarray]) -> int:
        '''
        Predict the digit for the utterance x.

        Parameters
        ----------
        x : list
            List of numpy arrays, where each element is a numpy array of shape (13, ~30).

        Returns
        -------
        predicted_digit : int
            Predicted digit.
        '''

        # If the mask hasn't been applied, apply it
        if x.shape[1]  != sum([1 for i in self.cepstral_mask if i]):
            x = x[:, self.cepstral_mask]

        predicted_means = self.predictions(x)
        # Divide by two as to get the digit rather than the GMM
        # The decimal precision is used to determine the calculated gender
        return np.argmax(predicted_means) / 2
    

    def test_accuracy(self, test_data: list[np.ndarray], verbose: bool = False) -> float:
        '''
        Compute the accuracy of the model on the test data.
        Assumes that test_data is actually the test data, as it uses the sizes from before to understand male/female distribution.

        Parameters
        ----------
        test_data : list
            List of numpy arrays, where each element is a numpy array of shape (13, ~30).

        verbose : bool
            Whether to print the accuracy for each digit.

        Returns
        -------
        accuracies : tuple
            Tuple of (overall_accuracy, digit_accuracies).
        '''

        # Again mask the cepstral coefficients by copying and slicing the data
        processed_test_data = []
        for i, digit in enumerate(test_data):
            digit_utterances = []
            for utterance in digit:
                # Apply the mask
                sliced = utterance[:, self.cepstral_mask]
                # Slice by male and female speakers
                m = sliced[:test_male_frames[i], :]
                f = sliced[test_male_frames[i]:, :]
                # Append both only if both have nonzero shape
                if m.shape[0] > 0:
                    digit_utterances.append(m)
                if f.shape[0] > 0:
                    digit_utterances.append(f)
            processed_test_data.append(digit_utterances)

        total_tested = 0
        correct = 0

        digit_accuracies = []

        # Loop through all different digits
        for digit_num, digit in enumerate(processed_test_data):
            digit_utterances_correct = 0
            total_digit_utterances = len(digit)

            # Loop through all utterances for the current digit
            for utterance in digit:
                predicted_digit = self.predict(utterance)
                # Int as to ignore gender
                if int(predicted_digit) == digit_num:
                    digit_utterances_correct += 1

            if verbose:
                print(f'Accuracy for digit {digit_num}: {digit_utterances_correct / total_digit_utterances}')

            digit_accuracies.append(digit_utterances_correct / total_digit_utterances)

            total_tested += total_digit_utterances
            correct += digit_utterances_correct

        if verbose:
            print(f'Overall accuracy: {correct / total_tested}')

        return correct / total_tested



## Hyperparameter Optimization

The hyperparameters in this problem are:
* Number of clusters in each GMM (5-15 approximately)
* Consideration of user-provided gender information (include or do not include)
* Which MFCC coefficients to use (all, or a subset)
* Covariance type of the GMMs (full, diagonal, spherical, or tied)

Determining the optimal hyperparameters is a difficult task. We will use a custom grid search to find the optimal hyperparameters.


In [26]:
import concurrent.futures
import numpy as np
import pandas as pd
from tqdm import tqdm


def perform_grid_search(train, test, num_samples=1000, cluster_range=(5, 16), covariance_types=['full', 'spherical', 'diag', 'tied'], max_mfcc_excluded=5):
    '''
    Perform a grid search over the parameter space and display the results.
    
    Parameters
    ----------
    train : list
        List of numpy arrays, where each element is the concatenation of all utterances for a given digit.
        
    test : list
        List of numpy arrays, where each element is a numpy array of shape (13, ~30).
        
    num_samples : int
        Number of samples to take from the parameter space.
        
    cluster_range : tuple
        Range of clusters to sample from.
        
    covariance_types : list
        List of covariance types to sample from.
        
    max_mfcc_excluded : int
        Maximum number of MFCCs to exclude.
        
        '''

    # Sample points from the parameter space
    num_clusters = np.random.randint(cluster_range[0], cluster_range[1], size=num_samples)
    covariances = np.random.randint(0, len(covariance_types), size=num_samples)

    # Generate MFCC masks
    # Each row is a mask and should have at most max_mfcc_excluded False values
    mfccs = []
    for _ in range(num_samples):
        mask = np.zeros((13,), dtype=bool)
        # Set all values to True
        mask[:] = True
        # 50% chance to exclude some MFCCs
        if np.random.randint(0, 2):
            # Determine how many False placement tries we have
            num_false = np.random.randint(0, max_mfcc_excluded + 1)
            # Generate num_false random indices
            mask[np.random.randint(0, 13, size=num_false)] = False
        mfccs.append(mask)

    mfccs = np.array(mfccs)

    # Create a dataframe to store the results
    df = pd.DataFrame({'Clusters': num_clusters, 'Covariance': covariances, 'MFCCs': mfccs.tolist()})
    df['accuracy'] = 0

    # Go through and replace the Covariance column with the actual covariance types
    df['Covariance'] = df['Covariance'].apply(lambda x: covariance_types[x])

    print(f'Searching {num_samples} points in the parameter space...')

    # Function to process each row concurrently
    def process_row(i):
        row = df.iloc[i]
        model = DigitClassifier(num_clusters=row['Clusters'], covariance_type=row['Covariance'], cepstral_mask=row['MFCCs'])
        model.fit(train)
        accuracy = model.test_accuracy(test)
        return accuracy

    # Concurrent processing of rows
    with concurrent.futures.ThreadPoolExecutor() as executor:
        accuracies = list(tqdm(executor.map(process_row, range(num_samples)), total=num_samples))

    # Assign accuracies to the dataframe
    df['Accuracy'] = accuracies

    # Sort the dataframe by accuracy
    df = df.sort_values(by='Accuracy', ascending=False)

    # Replace the MFCC column with 0s and 1s to make it fit better
    df['MFCCs'] = df['MFCCs'].apply(lambda x: [int(i) for i in x])

    display(df.head())


In [27]:
import concurrent.futures


def perform_genetic_algorithm(train, test, population_size=100, num_generations=10, elite_percentage=0.1,
                              cluster_range=(5, 16), covariance_types=['full', 'spherical', 'diag', 'tied'],
                              max_mfcc_excluded=5):
    '''
    Perform a genetic algorithm to find the best parameters for the digit classifier.

    Displays the top 5 results.

    Parameters
    ----------
    train : list
        List of numpy arrays, where each element is the concatenation of all utterances for a given digit.

    test : list
        List of numpy arrays, where each element is the concatenation of all utterances for a given digit.

    population_size : int
        Number of individuals in the population.

    num_generations : int
        Number of generations to run the algorithm for.

    elite_percentage : float
        Percentage of the population to select as elite individuals i.e. individuals that will be carried over to the next generation.

    cluster_range : tuple
        Range of clusters to use for the Gaussian Mixture Model.

    covariance_types : list
        List of covariance types to use for the Gaussian Mixture Model.

    max_mfcc_excluded : int
        Maximum number of MFCCs to exclude from the model.
    '''
    def generate_population():
        population = []
        for _ in range(population_size):
            num_clusters = np.random.randint(cluster_range[0], cluster_range[1])
            cov_type = np.random.choice(covariance_types)
            cepstral_mask = np.zeros((13,), dtype=bool)
            considers_gender = np.random.randint(0, 2)
            # Set all values to True
            cepstral_mask[:] = True
            # 50% chance to exclude some MFCCs
            if np.random.randint(0, 2):
                # Determine how many False placement tries we have
                num_false = np.random.randint(0, max_mfcc_excluded + 1)
                # Generate num_false random indices
                cepstral_mask[np.random.randint(0, 13, size=num_false)] = False
            population.append({'Clusters': num_clusters, 'Covariance': cov_type, 'MFCCs': cepstral_mask, 'Gender': considers_gender})
        return population

    def evaluate_population(pop):
        accuracies = []
        with concurrent.futures.ThreadPoolExecutor():
            for individual in tqdm(pop, desc='Evaluating population'):
                if individual['Gender']:
                    model = GenderDigitClassifier(num_clusters=individual['Clusters'],
                                                  covariance_type=individual['Covariance'],
                                                  cepstral_mask=individual['MFCCs'])
                else:
                    model = DigitClassifier(num_clusters=individual['Clusters'],
                                            covariance_type=individual['Covariance'],
                                            cepstral_mask=individual['MFCCs'])
                model.fit(train)
                accuracy = model.test_accuracy(test)
                accuracies.append(accuracy)
        return accuracies

    population = generate_population()

    for generation in range(num_generations):
        print(f"Generation {generation + 1}/{num_generations}")
        accuracies = evaluate_population(population)

        # Combine population with accuracies for sorting
        population_with_acc = list(zip(population, accuracies))
        population_with_acc.sort(key=lambda x: x[1], reverse=True)

        # Select elite individuals
        elite_count = int(elite_percentage * population_size)
        elite = [x[0] for x in population_with_acc[:elite_count]]

        # Create new population using elite individuals and random selection
        new_population = elite.copy()
        while len(new_population) < population_size:

            # Crossover
            parent1, parent2 = np.random.choice(elite, size=2)
            crossover_point = np.random.randint(1, len(parent1) - 1)
            child = {'Clusters': np.random.choice([parent1['Clusters'], parent2['Clusters']]),
                     'Covariance': np.random.choice([parent1['Covariance'], parent2['Covariance']]),
                     'MFCCs': np.concatenate([parent1['MFCCs'][:crossover_point], parent2['MFCCs'][crossover_point:]]),
                     'Gender': np.random.choice([parent1['Gender'], parent2['Gender']])}
            
            mutation_prob = 0.65

            # Mutate a MFCC inclusion
            if np.random.rand() < mutation_prob:
                mutation_point = np.random.randint(len(child['MFCCs']))
                child['MFCCs'][mutation_point] = 1 - child['MFCCs'][mutation_point]  # Flip bit for mutation

            # Mutate Clusters
            if np.random.rand() < mutation_prob:
                child['Clusters'] = np.random.randint(cluster_range[0], cluster_range[1])

            # Mutate Covariance
            if np.random.rand() < mutation_prob:
                child['Covariance'] = np.random.choice(covariance_types)

            # Mutate gender dependence
            if np.random.rand() < mutation_prob:
                child['Gender'] = not child['Gender']

            new_population.append(child)

        population = new_population

    # Final evaluation and sorting
    print('Final evaluation...')
    final_accuracies = evaluate_population(population)
    population_with_acc = list(zip(population, final_accuracies))
    population_with_acc.sort(key=lambda x: x[1], reverse=True)

    # Display top 10 results
    top_results = pd.DataFrame([{'Clusters': ind['Clusters'], 'Covariance': ind['Covariance'], 'MFCCs': ind['MFCCs'], 'Gender': ind['Gender'],
                                 'Accuracy': acc} for ind, acc in population_with_acc[:10]])
    
    # Replace the MFCC column with 0s and 1s to make it fit better
    top_results['MFCCs'] = top_results['MFCCs'].apply(lambda x: [int(i) for i in x])

    # Replace the gender column with 'uses' and 'does not use'
    top_results['Gender'] = top_results['Gender'].apply(lambda x: ['does not use', 'uses'][x])

    display(top_results)



perform_genetic_algorithm(X_TRAIN, TEST, population_size=200, num_generations=10, elite_percentage=0.15)

Generation 1/10


Evaluating population: 100%|██████████| 200/200 [1:18:48<00:00, 23.64s/it]


Generation 2/10


Evaluating population:  41%|████      | 82/200 [28:08<33:30, 17.04s/it]  /home/mbryant/anaconda3/lib/python3.11/site-packages/sklearn/mixture/_base.py:274: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
Evaluating population: 100%|██████████| 200/200 [1:12:28<00:00, 21.74s/it]


Generation 3/10


...

Generation 10/10


Evaluating population:  24%|██▎       | 47/200 [13:53<52:24, 20.55s/it]  /home/mbryant/anaconda3/lib/python3.11/site-packages/sklearn/mixture/_base.py:274: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
Evaluating population: 100%|██████████| 200/200 [57:13<00:00, 17.17s/it]


Final evaluation...


Evaluating population: 100%|██████████| 200/200 [54:54<00:00, 16.47s/it] 
/tmp/ipykernel_4320/3225794547.py:136: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  top_results['Gender'] = top_results['Gender'].apply(lambda x: ['does not use', 'uses'][x])


,Clusters,Covariance,MFCCs,Gender,Accuracy
0,5,diag,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]",uses,0.913636
1,7,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912727
2,7,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912727
3,6,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912273
4,6,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912273
5,6,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912273
6,6,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912273
7,6,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912273
8,6,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912273
9,6,diag,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",does not use,0.912273
